# Parse C Code
需要手动编译安装pet库.

In [1]:
import pet
import sys
import tempfile


def parse_code(source: str, func_name: str):
  with open("/tmp/parse_code.c", "w") as f:
    f.write(source)
  scop = pet.scop.extract_from_C_source("/tmp/parse_code.c", func_name)
  context = scop.get_context()
  schedule = scop.get_schedule()
  reads = scop.get_may_reads()
  writes = scop.get_may_writes()
  return (context, schedule, reads, writes)


context, schedule, reads, writes = parse_code("""
void rotate(int N, int A[N], int B[N])
{
#pragma scop
	for (int i = 0; i < N; ++i)
		A[i] = i == 0 ? B[N - 1] : B[i - 1];
#pragma endscop
}
""", "rotate")


In [2]:
print(context)

[N] -> {  : -2147483648 <= N <= 2147483647 }


In [3]:
print(schedule)

{ domain: "[N] -> { S_0[i] : 0 <= i < N }", child: { schedule: "[N] -> L_0[{ S_0[i] -> [(i)] }]" } }


In [4]:
print(reads)

[N] -> { S_0[i] -> B[-1 + i] : 0 < i < N; S_0[i = 0] -> B[-1 + N] : N > 0 }


In [5]:
print(writes)

[N] -> { S_0[i] -> A[i] : 0 <= i < N }
